# In this workshop we

- Build a RAG application on the FAQ database
- Make it agentic
- Learn about agentic search
- Give tools to our agents
- Use PydanticAI to make it easier

#### first lets install minsearch which we will turn it into agentic search

In [1]:
%pip install jupyter openai minsearch requests

Note: you may need to restart the kernel to use updated packages.


### Basic RAG

RAG consisits of 3 parts

- Search
- prompt
- LLM

``` python
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer
```
query -> get top k -> build prompt (using top k and other requirments as contexts) -> feed that to llm -> final search result.

In [2]:

import requests 
from minsearch import AppendableIndex

# get the documents 
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
        

# index them
index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)    

# build the search function

def search(query):
    
    boost = {'question': 3.0, 'section': 0.5}
    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results    

In [3]:
# building the prompt
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [4]:
# get our llm ready
from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-3.5-turbo-1106',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

##### Now that we have the RAG, let make it agentic

### Part 1: Agentic RAG

